In [ ]:
#best solution as of 11/5

In [ ]:
#to-do still
#change the files path to get data from git
#save this df in github so we can use it instead
#Figure out how to use process_street_actions for preflop

In [1]:
def get_hole_cards(game, street_indices, small_blind_name):
    """
    This function captures hole cards if they are available and correctly assigns them to players
    """
    hole_cards1 = None
    hole_cards2 = None

    # Get hole cards
    if 'shows' in str(game[street_indices[-1] + 1]):
        hole_cards1 = game[street_indices[-1] + 1][-11:-1]
        hole_cards2 = game[street_indices[-1] + 2][-11:-1]

    # Assign the hole cards correctly
    hole_cards1_player = game[street_indices[-1] + 1][0:game[street_indices[-1] + 1].index(' ') + 1]
    if hole_cards1_player == small_blind_name:
        sb_hole_cards = hole_cards1
        bb_hole_cards = hole_cards2
    else:
        sb_hole_cards = hole_cards2
        bb_hole_cards = hole_cards1

    return sb_hole_cards, bb_hole_cards

def left(s, amount):
    """
    This is unnecessary but I am using it anyway.
    """
    return s[:amount]


In [2]:
def process_street_actions(game, start_index, end_index):
    """
    This function loops over the actions that players made during the game and stores them in a list of tuples
    """
    street_list = []
    for i in range(start_index, end_index):
        if 'folds' in game[i]:
            thistuple = ('f', 0)
            street_list.append(thistuple)
        if ('calls' in game[i]) or ('bets' in game[i]) or ('raises' in game[i]):
            dollar = game[i].index('$')
            amount = game[i][dollar + 1:-2]
            first_space = game[i].index(' ')
            action = game[i][first_space + 1:first_space + 5]
            tuple_to_add = (action[0], amount)
            street_list.append(tuple_to_add)
        if 'checks' in game[i]:
            thistuple = ('k', 0)
            street_list.append(thistuple)
    return street_list

In [3]:
#from collections import Counter
#import numpy as np
import os
#from random import randint
#from collections import defaultdict
import pandas as pd
#import csv
from tqdm import tqdm
from IPython.core.interactiveshell import InteractiveShell # for all output 
InteractiveShell.ast_node_interactivity = "all"

files_path = [os.path.join(r,file) for r,d,f in os.walk(r"C:\Users\Peter\Desktop\Springboard\capstone1\Bought hands\unzipped") for file in f]


folded_pre=0
rows_list=[]

for f in tqdm(files_path):
    with open(f, 'r') as file1:
        lines = file1.readlines()
    file_name = os.path.basename(f)
    
    # Find indices of lines starting with '#Game'
    game_indices = [i for i, line in enumerate(lines) if line.strip().startswith('#Game')]
    
    # Extract game data based on indices
    games = [lines[game_indices[i-1]:game_indices[i]] for i in range(1, len(game_indices))]
  
    # Now, process each game individually
    for index, game in enumerate(games):
        try:
            game_number = game[0].strip()[11:] 
            small_blind_string_index = game[8].find('posts small blind')
            big_blind_string_index = game[9].find('posts big blind')    
            small_blind_name = game[8][0:small_blind_string_index] #with a trailing space
            big_blind_name = game[9][0:big_blind_string_index]
            i = 10 #this is where line **Dealing down cards ** is
            preflop=[]
            flop=[]
            turn=[]
            river=[]
            preflop_counter = 0
            flop_counter=0
            stack_sb_index = game[6].find('$')
            stack_bb_index = game[7].find('$')
            stack_sb = game[6][stack_sb_index+1:-3]
            stack_bb = game[7][stack_bb_index+1:-3]
            flop_cards=None
            turn_card=None
            river_card=None
            
#find indeces of flop, turn , river and summary

            street_indeces = []
            for index_in_game, line in enumerate(game):
                if '** Dealing flop' in line or '** Dealing turn' in line or '** Dealing river' in line:
                    street_indeces.append(index_in_game)
                elif line.startswith('** Summary **'):
                    street_indeces.append(index_in_game)
                    
#getting hole cards using the function            
            sb_hole_cards, bb_hole_cards = get_hole_cards(game, street_indeces, small_blind_name)
                                
#Getting preflop actions 

#currently working on runnign this by calling process_street_actions
            while not ((game[i][:13] == ('** Summary **')) or (game[i][0:18] == ('** Dealing flop **'))):
                if 'folds' in game[i]:
                    thistuple= ('f',0)
                    preflop.append(thistuple)
                if ('calls' in game[i]) or ('bets' in game[i]) or ('raises' in game[i]):
                    dollar = game[i].index('$')
                    amount = game[i][dollar+1:-2]
                    first_space = game[i].index(' ')
                    action = game[i][first_space+1:first_space+5]
                    tuple_to_add = (action[0],amount)
                    preflop.append(tuple_to_add)
                if 'checks' in game[i]:
                    thistuple= ('k',0) #k for check
                    preflop.append(thistuple)
                i=i+1
                preflop_counter +=1
            
#Getting flop cards and actions
            if len(street_indeces)>1:
                flop_cards = [game[street_indeces[0]][-13:-11],game[street_indeces[0]][-9:-7],game[street_indeces[0]][-5:-3]]
                flop = process_street_actions(game, street_indeces[0], street_indeces[1])
          
#Getting turn cards and actions
            if len(street_indeces)>2:
                turn_card = game[street_indeces[1]][-5:-3]
                turn = process_street_actions(game, street_indeces[1], street_indeces[2])
             
#Getting river cards and actions
            if len(street_indeces)>3:
                river_card = game[street_indeces[2]][-5:-3]
                river=process_street_actions(game, street_indeces[2], street_indeces[3])
           
            remove_folded_pre =False  #add this parameter to drop hands that were immediately folded by SB to reduce the DF
            if preflop[0][0][0]=='f':
                folded_pre+=1
                remove_folded_pre=True 
                
#Create a dictionary with all the data per game
            river = river if river else None #if empty change to None to save space
            turn = turn if turn else None
            flop = flop if flop else None

            game_row  = {'Game ID':game_number,'File':file_name ,'Player SB':small_blind_name,
                         'Player BB':big_blind_name, 'Preflop actions':preflop, 
                         'Flop actions':flop, 'Turn actions':turn,'River actions':river, 
                         'Flop':flop_cards, 'Turn':turn_card, 'River':river_card, 'SB stack':stack_sb, 
                         'BB stack': stack_bb, 'SB cards': sb_hole_cards, 'BB cards':bb_hole_cards, 
                         'Folded pre':remove_folded_pre}
            rows_list.append(game_row)

        except:
            pass


100%|██████████████████████████████████████████████████████████████████████████| 28971/28971 [00:19<00:00, 1524.65it/s]


In [4]:
df = pd.DataFrame(rows_list)

# Now df contains the data in a structured format
df

,Game ID,File,Player SB,Player BB,Preflop actions,Flop actions,Turn actions,River actions,Flop,Turn,River,SB stack,BB stack,SB cards,BB cards,Folded pre
0,502874582,Arezzo-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,Oracool1,sirstadiljus,"[(f, 0)]",None,None,None,None,None,None,1.22,2.01,None,None,True
1,603604223,Asmara-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,ArcticWin,Ceaban,"[(c, 0.01), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[(b, 0.04), (c, 0.04)]","[5c, 9c, 5s]",7h,2c,2,1,"[ Qh, Tc ]","[ 6h, 2h ]",False
2,603604231,Asmara-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,Ceaban,ArcticWin,"[(f, 0)]",None,None,None,None,None,None,1.94,1.06,None,None,True
3,603604236,Asmara-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,ArcticWin,Ceaban,"[(f, 0)]",None,None,None,None,None,None,1.95,1.05,None,None,True
4,603604241,Asmara-0.01-0.02-USD-NoLimitHoldem-Pacific-6-4...,Ceaban,ArcticWin,"[(f, 0)]",None,None,None,None,None,None,1.94,1.06,None,None,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520800,553224385,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,Sensei.Dbb,roghnov,"[(c, 0.01), (k, 0)]","[(k, 0), (b, 0.02), (f, 0)]",None,None,"[Kc, 5d, 5s]",None,None,1.02,0.98,None,None,False
520801,553224389,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,roghnov,Sensei.Dbb,"[(r, 0.03), (f, 0)]",None,None,None,None,None,None,1,1,None,None,False
520802,553224821,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,mikvell,John_Efapi,"[(c, 0.01), (k, 0)]","[(b, 0.04), (c, 0.04)]","[(k, 0), (b, 0.08), (f, 0)]",None,"[9h, 6h, 7h]",Ks,None,3.36,1.90,None,None,False
520803,553224834,Corrientes-0.01-0.02-USD-NoLimitHoldem-Pacific...,John_Efapi,mikvell,"[(r, 0.05), (c, 0.04)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[(k, 0), (k, 0)]","[9d, Qc, 5d]",7d,Ac,3.42,1.84,"[ 3d, 3s ]","[ 4h, Ah ]",False
